## Requirements

In [107]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import csv
import os
import logging
import argparse
import random
from tqdm import tqdm, trange
import json

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, f1_score
from plot_confusion_matrix import plot_confusion_matrix

from models.stacked_debert_dae.tokenization import BertTokenizer
from models.stacked_debert_dae.modeling import BertForSequenceClassification
from models.stacked_debert_dae import BertAdam
from models.stacked_debert_dae import PYTORCH_PRETRAINED_BERT_CACHE
from models.stacked_debert_dae.denoising_mlp import AutoEncoder


logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

### Data processing functions

In [108]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding='utf-8') as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                lines.append(line)
            return lines


class SentenceClassificationProcessor(DataProcessor):
    """Processor for Intent classification dataset."""

    def __init__(self, labels_array):
        self.labels_array = labels_array

    def get_train_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "test.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return self.labels_array

    def _create_examples(self, lines, set_type):
        """
        Creates examples for the training and dev sets.
        Add target sentence for AutoEncoder.
        """
        examples, examples_target = [], []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[0]
            label = line[1]
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
            if len(line) == 4:
                target_a = line[3]
                examples_target.append(
                    InputExample(guid=guid, text_a=target_a, text_b=None, label=label))
        return examples, examples_target


def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        label_id = label_map[example.label]
        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %d)" % (example.label, label_id))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id))
    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

### Other functions

In [109]:
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)


def warmup_linear(x, warmup=0.002):
    if x < warmup:
        return x/warmup
    return 1.0 - x


## Model

### 1. Evaluation functions

In [110]:
# ======== EARLY STOPPING ========
def evaluate_original_bert_model(args, model, device, eval_dataloader, eval_examples_dataloader, tr_loss, nb_tr_steps, global_step, batches=None):
    # Batches: None if eval and small number if eval is being used in train
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    predicted = []
    target = []
    predicted_mismatch, target_mismatch, text_mismatch = [], [], []
    counter = 0
    for (input_ids, input_mask, segment_ids, label_ids), ex in zip(eval_dataloader, eval_examples_dataloader):
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)

        with torch.no_grad():
            _, tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
            _, logits = model(input_ids, segment_ids, input_mask)

        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()
        tmp_eval_accuracy = accuracy(logits, label_ids)

        logits_max = np.argmax(logits, axis=1)
        for pred, lab, e in zip(logits_max, label_ids, ex):
            predicted.append(pred)
            target.append(lab)
            if pred != lab:
                predicted_mismatch.append(pred)
                target_mismatch.append(lab)
                text_mismatch.append(e)

        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy

        nb_eval_examples += input_ids.size(0)
        nb_eval_steps += 1
        if batches is not None and counter >= batches:
            break
        counter += 1

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_examples
    loss = tr_loss / nb_tr_steps if args.do_train else None
    result = {'eval_loss': eval_loss,
              'eval_accuracy': eval_accuracy,
              'global_step': global_step,
              'loss': loss}
    return eval_loss, result, target, predicted, text_mismatch, target_mismatch, predicted_mismatch


def evaluate_model(args, model, device, eval_dataloader, eval_examples_dataloader, tr_loss, nb_tr_steps, global_step,
                   autoencoder, model_second_stack, batches=None):
    # Batches: None if eval and small number if eval is being used in train
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    predicted = []
    target = []
    predicted_mismatch, target_mismatch, text_mismatch = [], [], []
    counter = 0
    for (input_ids, input_mask, segment_ids, label_ids), ex in zip(eval_dataloader, eval_examples_dataloader):
        # tqdm(zip(eval_dataloader, eval_examples_dataloader), desc="Evaluating"):
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)

        with torch.no_grad():
            encoded_layers, tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
            # _, out_decoder = autoencoder(encoded_layers)
            # encoded_layers = out_decoder
            encoded_layers = encoded_layers.view(-1, 768)
            _, out_decoder = autoencoder(encoded_layers)
            encoded_layers = out_decoder.view(args.eval_batch_size, -1, 768)
            _, tmp_eval_loss = model_second_stack(input_ids, segment_ids, input_mask, label_ids,
                                                  words_embeddings=encoded_layers)
            _, logits = model_second_stack(input_ids, segment_ids, input_mask, words_embeddings=encoded_layers)

        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()
        tmp_eval_accuracy = accuracy(logits, label_ids)

        logits_max = np.argmax(logits, axis=1)
        for pred, lab, e in zip(logits_max, label_ids, ex):
            predicted.append(pred)
            target.append(lab)
            if pred != lab:
                predicted_mismatch.append(pred)
                target_mismatch.append(lab)
                text_mismatch.append(e)

        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy

        nb_eval_examples += input_ids.size(0)
        nb_eval_steps += 1

        if batches is not None and counter >= batches:
            break
        counter += 1

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_examples
    loss = tr_loss / nb_tr_steps if args.do_train or args.do_train_second_layer else None
    result = {'eval_loss': eval_loss,
              'eval_accuracy': eval_accuracy,
              'global_step': global_step,
              'loss': loss}
    return eval_loss, result, target, predicted, text_mismatch, target_mismatch, predicted_mismatch
# ================================

### 2. Training functions: BERT, denoising transformer layer

In [111]:
def first_bert_layer(train_examples, num_train_steps, args, tokenizer, label_list, num_labels, device, n_gpu,
                     eval_dataloader, eval_examples_dataloader):

    # Prepare model
    model = BertForSequenceClassification.from_pretrained(args.bert_model,
                                                          cache_dir=PYTORCH_PRETRAINED_BERT_CACHE / 'distributed_{}'.format(
                                                              args.local_rank),
                                                          num_labels=num_labels)
    if args.fp16:
        model.half()
    model.to(device)
    if args.local_rank != -1:
        try:
            from apex.parallel import DistributedDataParallel as DDP
        except ImportError:
            raise ImportError(
                "Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

        model = DDP(model)
    elif n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Prepare optimizer
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    t_total = num_train_steps
    if args.local_rank != -1:
        t_total = t_total // torch.distributed.get_world_size()
    if args.fp16:
        try:
            from apex.optimizers import FP16_Optimizer
            from apex.optimizers import FusedAdam
        except ImportError:
            raise ImportError(
                "Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

        optimizer = FusedAdam(optimizer_grouped_parameters,
                              lr=args.learning_rate,
                              bias_correction=False,
                              max_grad_norm=1.0)
        if args.loss_scale == 0:
            optimizer = FP16_Optimizer(optimizer, dynamic_loss_scale=True)
        else:
            optimizer = FP16_Optimizer(optimizer, static_loss_scale=args.loss_scale)

    else:
        optimizer = BertAdam(optimizer_grouped_parameters,
                             lr=args.learning_rate,
                             warmup=args.warmup_proportion,
                             t_total=t_total)

    global_step = 0
    nb_tr_steps = 0
    tr_loss = 0
    # EARLY STOPPING
    last_eval_loss = float("inf")
    if args.do_train:
        train_features = convert_examples_to_features(
            train_examples, label_list, args.max_seq_length, tokenizer)
        logger.info("***** Running training *****")
        logger.info("  Num examples = %d", len(train_examples))
        logger.info("  Batch size = %d", args.train_batch_size)
        logger.info("  Num steps = %d", num_train_steps)
        all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
        train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
        if args.local_rank == -1:
            train_sampler = RandomSampler(train_data)
        else:
            train_sampler = DistributedSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)

        model.train()
        for _ in trange(int(args.num_train_epochs), desc="Epoch"):
            tr_loss = 0
            nb_tr_examples, nb_tr_steps = 0, 0
            for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
                batch = tuple(t.to(device) for t in batch)
                input_ids, input_mask, segment_ids, label_ids = batch
                _, loss = model(input_ids, segment_ids, input_mask, label_ids)
                if n_gpu > 1:
                    loss = loss.mean()  # mean() to average on multi-gpu.
                if args.gradient_accumulation_steps > 1:
                    loss = loss / args.gradient_accumulation_steps

                if args.fp16:
                    optimizer.backward(loss)
                else:
                    loss.backward()

                tr_loss += loss.item()
                nb_tr_examples += input_ids.size(0)
                nb_tr_steps += 1
                if (step + 1) % args.gradient_accumulation_steps == 0:
                    # modify learning rate with special warm up BERT uses
                    lr_this_step = args.learning_rate * warmup_linear(global_step / t_total, args.warmup_proportion)
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = lr_this_step
                    optimizer.step()
                    optimizer.zero_grad()
                    global_step += 1

                    # ======== EARLY STOPPING ========
                    eval_loss, _, _, _, _, _, _ = evaluate_original_bert_model(args, model, device, eval_dataloader,
                                                                               eval_examples_dataloader, tr_loss,
                                                                               nb_tr_steps, global_step, batches=4)
                    if args.save_best_model:
                        # Save a trained model - EARLY STOPPING
                        if eval_loss <= last_eval_loss:
                            model_to_save = model.module if hasattr(model,
                                                                    'module') else model  # Only save the model it-self
                            output_model_file = os.path.join(args.output_dir_first_layer, "pytorch_model.bin")
                            torch.save(model_to_save.state_dict(), output_model_file)
                            last_eval_loss = eval_loss
                    # ================================

    # Save a trained model
    if not args.save_best_model:
        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        output_model_file = os.path.join(args.output_dir_first_layer, "pytorch_model.bin")
        torch.save(model_to_save.state_dict(), output_model_file)

    return model, global_step, nb_tr_steps, tr_loss

def autoencoder_layer(train_examples, train_examples_target, model_first_layer, args, tokenizer, label_list, device, n_gpu):

    # Prepare model
    autoencoder = AutoEncoder()

    if args.fp16:
        autoencoder.half()
    autoencoder.to(device)
    if args.local_rank != -1:
        try:
            from apex.parallel import DistributedDataParallel as DDP
        except ImportError:
            raise ImportError(
                "Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

        autoencoder = DDP(autoencoder)
    elif n_gpu > 1:
        autoencoder = torch.nn.DataParallel(autoencoder)

    # Prepare optimizer
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001, weight_decay=1e-5)
    criterion = torch.nn.MSELoss()

    if args.do_train_second_layer:
        train_features = convert_examples_to_features(
            train_examples, label_list, args.max_seq_length, tokenizer)
        logger.info("***** Running training *****")
        logger.info("  Num examples = %d", len(train_examples))
        logger.info("  Batch size = %d", args.train_batch_size)
        all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)

        train_features = convert_examples_to_features(
            train_examples_target, label_list, args.max_seq_length, tokenizer)
        logger.info("***** Running training *****")
        logger.info("  Num examples = %d", len(train_examples_target))
        logger.info("  Batch size = %d", args.train_batch_size)
        all_input_ids_target = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
        all_input_mask_target = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
        all_segment_ids_target = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
        all_label_ids_target = torch.tensor([f.label_id for f in train_features], dtype=torch.long)

        train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids,
                                   all_input_ids_target, all_input_mask_target, all_segment_ids_target, all_label_ids_target)
        if args.local_rank == -1:
            train_sampler = RandomSampler(train_data)
        else:
            train_sampler = DistributedSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)

        autoencoder.train()
        for _ in trange(int(args.num_train_epochs_autoencoder), desc="Epoch"):
            tr_loss = 0
            nb_tr_examples, nb_tr_steps = 0, 0
            for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
                batch = tuple(t.to(device) for t in batch)
                input_ids, input_mask, segment_ids, label_ids, input_ids_target, input_mask_target, segment_ids_target, label_ids_target = batch
                # BERT model on INcomplete data
                encoded_layers, _ = model_first_layer(input_ids, segment_ids, input_mask, label_ids)
                encoded_layers = encoded_layers.view(-1, 768)
                # BERT model on complete data
                encoded_layers_complete, _ = model_first_layer(input_ids_target, segment_ids_target, input_mask_target, label_ids_target)
                # Denoising Autoencoder trained on noisy input and clean output_cnn_hter_cr
                encoded_layers_complete = encoded_layers_complete.view(-1, 768)
                _, out_dec = autoencoder(encoded_layers)
                loss = criterion(out_dec, encoded_layers_complete)
                if n_gpu > 1:
                    loss = loss.mean()  # mean() to average on multi-gpu.
                if args.gradient_accumulation_steps > 1:
                    loss = loss / args.gradient_accumulation_steps

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                tr_loss += loss.item()
                nb_tr_examples += input_ids.size(0)
                nb_tr_steps += 1

    return autoencoder


def second_bert_layer(train_examples, num_train_steps, model_first_layer, autoencoder, args, tokenizer, label_list,
                      num_labels, device, n_gpu, eval_dataloader, eval_examples_dataloader):

    # Prepare model
    model = BertForSequenceClassification.from_pretrained(args.bert_model,
                                                          cache_dir=PYTORCH_PRETRAINED_BERT_CACHE / 'distributed_{}'.format(
                                                              args.local_rank),
                                                          num_labels=num_labels)
    if args.fp16:
        model.half()
    model.to(device)
    if args.local_rank != -1:
        try:
            from apex.parallel import DistributedDataParallel as DDP
        except ImportError:
            raise ImportError(
                "Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

        model = DDP(model)
    elif n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Prepare optimizer
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    t_total = num_train_steps
    if args.local_rank != -1:
        t_total = t_total // torch.distributed.get_world_size()
    if args.fp16:
        try:
            from apex.optimizers import FP16_Optimizer
            from apex.optimizers import FusedAdam
        except ImportError:
            raise ImportError(
                "Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

        optimizer = FusedAdam(optimizer_grouped_parameters,
                              lr=args.learning_rate,
                              bias_correction=False,
                              max_grad_norm=1.0)
        if args.loss_scale == 0:
            optimizer = FP16_Optimizer(optimizer, dynamic_loss_scale=True)
        else:
            optimizer = FP16_Optimizer(optimizer, static_loss_scale=args.loss_scale)

    else:
        optimizer = BertAdam(optimizer_grouped_parameters,
                             lr=args.learning_rate,
                             warmup=args.warmup_proportion,
                             t_total=t_total)

    global_step = 0
    nb_tr_steps = 0
    tr_loss = 0
    # EARLY STOPPING
    last_eval_loss = float("inf")
    if args.do_train_second_layer:
        train_features = convert_examples_to_features(
            train_examples, label_list, args.max_seq_length, tokenizer)
        logger.info("***** Running training *****")
        logger.info("  Num examples = %d", len(train_examples))
        logger.info("  Batch size = %d", args.train_batch_size)
        logger.info("  Num steps = %d", num_train_steps)
        all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
        train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
        if args.local_rank == -1:
            train_sampler = RandomSampler(train_data)
        else:
            train_sampler = DistributedSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)

        model.train()
        for _ in trange(int(args.num_train_epochs), desc="Epoch"):
            tr_loss = 0
            nb_tr_examples, nb_tr_steps = 0, 0
            for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
                batch = tuple(t.to(device) for t in batch)
                input_ids, input_mask, segment_ids, label_ids = batch
                encoded_layers, loss = model_first_layer(input_ids, segment_ids, input_mask, label_ids)
                # AutoEncoder
                encoded_layers = encoded_layers.view(-1, 768)
                _, out_decoder = autoencoder(encoded_layers)
                encoded_layers = out_decoder.view(args.train_batch_size, -1, 768)
                _, loss = model(input_ids, segment_ids, input_mask, label_ids, words_embeddings=encoded_layers)
                if n_gpu > 1:
                    loss = loss.mean()  # mean() to average on multi-gpu.
                if args.gradient_accumulation_steps > 1:
                    loss = loss / args.gradient_accumulation_steps

                if args.fp16:
                    optimizer.backward(loss)
                else:
                    loss.backward()

                tr_loss += loss.item()
                nb_tr_examples += input_ids.size(0)
                nb_tr_steps += 1
                if (step + 1) % args.gradient_accumulation_steps == 0:
                    # modify learning rate with special warm up BERT uses
                    lr_this_step = args.learning_rate * warmup_linear(global_step / t_total, args.warmup_proportion)
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = lr_this_step
                    optimizer.step()
                    optimizer.zero_grad()
                    global_step += 1

                    # ======== EARLY STOPPING ========
                    eval_loss, _, _, _, _, _, _ = evaluate_model(args, model_first_layer, device, eval_dataloader,
                                                                 eval_examples_dataloader, tr_loss, nb_tr_steps,
                                                                 global_step, autoencoder, model, batches=4)
                    if args.save_best_model:
                        # Save a trained model - EARLY STOPPING
                        if eval_loss <= last_eval_loss:
                            model_to_save = model.module if hasattr(model,
                                                                    'module') else model  # Only save the model it-self
                            output_model_file = os.path.join(args.output_dir, "pytorch_model.bin")
                            torch.save(model_to_save.state_dict(), output_model_file)
                            last_eval_loss = eval_loss
                    # ================================

    # Save a trained model
    model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
    output_model_file = os.path.join(args.output_dir, "pytorch_model.bin")
    torch.save(model_to_save.state_dict(), output_model_file)

    return model, global_step, nb_tr_steps, tr_loss

### 3. Main function
Example from gtts-witai (TTS-STT) Chatbot dataset

#### 3.1 - Parser

In [112]:
    # ipykernel_launcher.py: the following arguments are required: --data_dir, --bert_model, --task_name, --output_dir, --output_dir_first_layer
    DATA_DIR = "./data/intent_data/stterror_data/chatbot/gtts_witai/"
    BERT_MODEL = "bert-base-uncased"
    TASK_NAME = "chatbot_intent"
    OUTPUT_DIR = "./results/results_stacked_debert_STTerror"
    SEED, BS_TRAIN, BS_EVAL, EPOCH, EPOCH_AE = [1, 8, 1, 3, 100]
    EPOCH_1st = EPOCH
    OUTPUT_DIR_1st_LAYER = OUTPUT_DIR + "/chatbot/gtts_witai/bs" + str(BS_TRAIN) +"_epae" + str(EPOCH_AE) + "/chatbot_ep" + str(EPOCH_1st) + "_bs" + str(BS_TRAIN) + "_seed" + str(SEED) + "_first_layer_epae" + str(EPOCH_AE) + "/"
    OUTPUT_DIR_2nd_LAYER = OUTPUT_DIR + "/chatbot/gtts_witai/bs" + str(BS_TRAIN) +"_epae" + str(EPOCH_AE) + "/chatbot_ep" + str(EPOCH_1st) + "_bs" + str(BS_TRAIN) + "_seed" + str(SEED) + "_second_layer_epae" + str(EPOCH_AE) + "/"
    
    import easydict
    args = easydict.EasyDict({
        ## Required parameters
        "data_dir": DATA_DIR, # The input data dir. Should contain the .tsv files (or other data files) for the task.
        "bert_model": BERT_MODEL, # Bert pre-trained model selected in the list: bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese.
        "task_name": TASK_NAME, # The name of the task to train
        "output_dir": OUTPUT_DIR_2nd_LAYER, # The output_cnn_hter_cr directory where the model predictions and checkpoints will be written.
        "output_dir_first_layer": OUTPUT_DIR_1st_LAYER, # The output_cnn_hter_cr directory where the model predictions and checkpoints will be written.
        "max_noise": 0.1,
        ## Other parameters
        "max_seq_length": 128, # The maximum total input sequence length after WordPiece tokenization. Sequences longer than this will be truncated, and sequences shorter than this will be padded.
        "do_train": True, # Whether to train first BERT layer
        "do_train_second_layer": True, # Whether to train 2nd BERT (transformers)
        "do_train_autoencoder": True, # Whether to train the denoinsing layer
        "do_eval": True, # Whether to run eval on the dev set
        "do_lower_case": True, # Set this flag if you are using an uncased model
        "train_batch_size": BS_TRAIN, # Total batch size for training
        "eval_batch_size": BS_EVAL, # Total batch size for eval
        "learning_rate": 5e-5, # The initial learning rate for Adam
        "num_train_epochs": EPOCH, # Total number of training epochs to perform
        "num_train_epochs_autoencoder": EPOCH_AE, # Total number of training epochs to perform
        "warmup_proportion": 0.1, # Proportion of training to perform linear learning rate warmup for. E.g., 0.1 = 10%% of training
        "no_cuda": True, # Whether not to use CUDA when available
        "save_best_model": True, # Whether to do early stopping or not
        "local_rank": -1, # local_rank for distributed training on gpus
        "seed": SEED, # random seed for initialization
        "gradient_accumulation_steps": 1, # Number of updates steps to accumulate before performing a backward/update pass
        "fp16": True, # Whether to use 16-bit float precision instead of 32-bit
        "loss_scale": 0, # Loss scaling to improve fp16 numeric stability. Only used when fp16 set to True. 0 (default value): dynamic loss scaling. Positive power of 2: static loss scaling value.
        })

### 3.2 - Variables

In [113]:
    processors = {
        "chatbot_intent": SentenceClassificationProcessor,
        "sentiment140_sentiment": SentenceClassificationProcessor,
    }

    num_labels_task = {
        "chatbot_intent": 2,
        "sentiment140_sentiment": 2,
    }

    labels_array = {
        "chatbot_intent": ["0", "1"],
        "sentiment140_sentiment": ["0", "1"],
    }

    labels_array_int = {
        "chatbot_intent": [0, 1],
        "sentiment140_sentiment": [0, 1],
    }

    if args.local_rank == -1 or args.no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        n_gpu = torch.cuda.device_count()
    else:
        torch.cuda.set_device(args.local_rank)
        device = torch.device("cuda", args.local_rank)
        n_gpu = 1
        # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.distributed.init_process_group(backend='nccl')
    logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
        device, n_gpu, bool(args.local_rank != -1), args.fp16))

    if args.gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                            args.gradient_accumulation_steps))

    args.train_batch_size = int(args.train_batch_size / args.gradient_accumulation_steps)

    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

    if not args.do_train and not args.do_eval and not args.do_train_second_layer:
        raise ValueError("At least one of `do_train` or `do_eval` must be True.")

    if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train_second_layer:
        raise ValueError("Output directory ({}) already exists and is not empty.".format(args.output_dir))
    os.makedirs(args.output_dir, exist_ok=True)

    if os.path.exists(args.output_dir_first_layer) and os.listdir(args.output_dir_first_layer) and args.do_train:
        raise ValueError("Output directory ({}) already exists and is not empty.".format(args.output_dir_first_layer))
    os.makedirs(args.output_dir_first_layer, exist_ok=True)

    task_name = args.task_name.lower()

    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))

    processor = processors[task_name](labels_array[task_name])
    num_labels = num_labels_task[task_name]
    label_list = processor.get_labels()

    tokenizer = BertTokenizer.from_pretrained(args.bert_model, do_lower_case=args.do_lower_case)

    train_examples = None
    train_examples_target = None
    num_train_steps = None
    if args.do_train or args.do_train_second_layer or args.do_train_autoencoder:
        train_examples, train_examples_target = processor.get_train_examples(args.data_dir)
        num_train_steps = int(
            len(train_examples) / args.train_batch_size / args.gradient_accumulation_steps * args.num_train_epochs)

    # ======== EARLY STOPPING ========
    # Load evaluation dataset for use in early stopping
    # Eval dataloader
    eval_examples, _ = processor.get_dev_examples(args.data_dir)
    eval_features = convert_examples_to_features(
        eval_examples, label_list, args.max_seq_length, tokenizer)
    logger.info("***** Running evaluation *****")
    logger.info("  Num examples = %d", len(eval_examples))
    logger.info("  Batch size = %d", args.eval_batch_size)
    all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
    eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    # Run prediction for full data
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # Get texts
    text = []
    for e in eval_examples:
        text.append(e.text_a)
    eval_examples_dataloader = DataLoader(text, shuffle=False, batch_size=args.eval_batch_size)
    # ================================

08/21/2020 11:58:32 - INFO - __main__ -   device: cpu n_gpu: 1, distributed training: False, 16-bits training: True
08/21/2020 11:58:32 - INFO - models.stacked_debert_dae.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/ceslea/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
08/21/2020 11:58:32 - INFO - __main__ -   LOOKING AT ./data/intent_data/stterror_data/chatbot/gtts_witai/train.tsv
08/21/2020 11:58:32 - INFO - __main__ -   *** Example ***
08/21/2020 11:58:32 - INFO - __main__ -   guid: dev-1
08/21/2020 11:58:32 - INFO - __main__ -   tokens: [CLS] when is the next bus from ga ##rch ##ing for ##sch ##ung ##sz ##ent ##rum . [SEP]
08/21/2020 11:58:32 - INFO - __main__ -   input_ids: 101 2043 2003 1996 2279 3902 2013 11721 11140 2075 2005 11624 5575 17112 4765 6824 1012 102 0 0 0 0 0 0 0 0 

In [114]:
    # Train 1st BERT
    global_step, nb_tr_steps, tr_loss = 0, 0, 0
    if args.do_train:
        model, global_step, nb_tr_steps, tr_loss = first_bert_layer(train_examples, num_train_steps, args, tokenizer,
                                                                    label_list, num_labels, device, n_gpu,
                                                                    eval_dataloader, eval_examples_dataloader)

    # Load a trained model that you have fine-tuned
    output_model_file = os.path.join(args.output_dir_first_layer, "pytorch_model.bin")
    model_state_dict = torch.load(output_model_file)
    model_first_layer = BertForSequenceClassification.from_pretrained(args.bert_model, state_dict=model_state_dict,
                                                                      num_labels=num_labels)
    model_first_layer.to(device)

08/21/2020 11:58:33 - INFO - models.stacked_debert_dae.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/ceslea/.pytorch_pretrained_bert/distributed_-1/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
08/21/2020 11:58:33 - INFO - models.stacked_debert_dae.modeling -   extracting archive file /home/ceslea/.pytorch_pretrained_bert/distributed_-1/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmp6hbb6xjm
08/21/2020 11:58:36 - INFO - models.stacked_debert_dae.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_

ImportError: Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.

In [ ]:
    # Train autoencoder
    # TODO: Early Stopping for AutoEncoder
    if args.do_train_autoencoder:
        autoencoder = autoencoder_layer(train_examples, train_examples_target, model_first_layer, args, tokenizer, label_list, device, n_gpu)

        # Save a trained model
        model_to_save = autoencoder.module if hasattr(autoencoder, 'module') else autoencoder  # Only save the model it-self
        output_model_file = os.path.join(args.output_dir, "autoencoder_pytorch_model.bin")
        torch.save(model_to_save.state_dict(), output_model_file)

In [ ]:
    # Train 2nd BERT layer
    if args.do_train_second_layer:
        # Load trained autoencoder
        output_model_file_ae = os.path.join(args.output_dir, "autoencoder_pytorch_model.bin")
        model_state_dict_ae = torch.load(output_model_file_ae)
        autoencoder = AutoEncoder()
        autoencoder.load_state_dict(model_state_dict_ae)
        autoencoder.to(device)

        model_second_stack, global_step, nb_tr_steps, tr_loss = second_bert_layer(train_examples, num_train_steps,
                                                                                  model_first_layer, autoencoder, args,
                                                                                  tokenizer, label_list, num_labels,
                                                                                  device, n_gpu, eval_dataloader,
                                                                                  eval_examples_dataloader)

In [ ]:
    # ======== LOAD TEST DATA ========
    # Test dataloader
    test_examples, _ = processor.get_test_examples(args.data_dir)
    test_features = convert_examples_to_features(test_examples, label_list, args.max_seq_length, tokenizer)
    logger.info("***** Loading test data *****")
    logger.info("  Num examples = %d", len(test_examples))
    logger.info("  Batch size = %d", args.eval_batch_size)
    all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in test_features], dtype=torch.long)
    test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    # Run prediction for full data
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=args.eval_batch_size)

    # Get texts
    text = []
    for e in test_examples:
        text.append(e.text_a)
    test_examples_dataloader = DataLoader(text, shuffle=False, batch_size=args.eval_batch_size)

    # ======== TEST with TEST DATA ========
    # Load a trained model that you have fine-tuned
    model_state_dict = torch.load(os.path.join(args.output_dir_first_layer, "pytorch_model.bin"))
    model = BertForSequenceClassification.from_pretrained(args.bert_model, state_dict=model_state_dict, num_labels=num_labels)
    model.to(device)

    # Load trained autoencoder
    output_model_file_ae = os.path.join(args.output_dir, "autoencoder_pytorch_model.bin")
    model_state_dict_ae = torch.load(output_model_file_ae)
    autoencoder = AutoEncoder()
    autoencoder.load_state_dict(model_state_dict_ae)
    autoencoder.to(device)

    # Load second layer
    model_state_dict_second_stack = torch.load(os.path.join(args.output_dir, "pytorch_model.bin"))
    model_second_stack = BertForSequenceClassification.from_pretrained(args.bert_model,
                                                                       state_dict=model_state_dict_second_stack,
                                                                       num_labels=num_labels)
    model_second_stack.to(device)

    if args.do_eval and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
        model.eval()
        eval_loss, result, target, predicted, text_mismatch, target_mismatch, predicted_mismatch = \
            evaluate_model(args, model, device, test_dataloader, test_examples_dataloader, tr_loss, nb_tr_steps,
                           global_step, autoencoder, model_second_stack, batches=None)

        # Save model information
        # CHECK IF target and predicted need to be array
        labels = labels_array_int[task_name]
        result['precision_macro'], result['recall_macro'], result['f1_macro'], support =\
            precision_recall_fscore_support(target, predicted, average='macro', labels=labels)
        result['precision_micro'], result['recall_micro'], result['f1_micro'], support =\
            precision_recall_fscore_support(target, predicted, average='micro', labels=labels)
        result['precision_weighted'], result['recall_weighted'], result['f1_weighted'], support =\
            precision_recall_fscore_support(target, predicted, average='weighted', labels=labels)
        result['confusion_matrix'] = confusion_matrix(target, predicted, labels=labels).tolist()

        output_eval_filename = "eval_results"
        if not args.do_train:
            output_eval_filename = "eval_results_test"

        # target_asarray = np.asarray(target)
        # predicted_asarray = np.asarray(predicted)
        classes = np.asarray(labels_array[task_name])
        classes_idx = None
        if 'webapplications' in task_name:
            classes = np.asarray(['0', '1', '3', '4', '5', '6', '7'])  # there is no class 2 in test
            classes_idx = np.asarray([0, 1, 2, 3, 4, 5, 6])
        target_asarray = np.asarray(target)
        predicted_asarray = np.asarray(predicted)
        ax, fig = plot_confusion_matrix(target_asarray, predicted_asarray, classes=classes,
                                        normalize=True, title='Normalized confusion matrix', rotate=False,
                                        classes_idx=classes_idx)
        fig.savefig(os.path.join(args.output_dir, output_eval_filename + "_confusion.png"))

        output_eval_file = os.path.join(args.output_dir, output_eval_filename + ".json")
        with open(output_eval_file, "w") as writer:
            json.dump(result, writer, indent=2)

        output_eval_file = os.path.join(args.output_dir, output_eval_filename + ".txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results *****")
            for key in sorted(result.keys()):
                logger.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key])))

        # Example sentences and original and predicted labels
        output_eval_filename = "eval_examples"
        if not args.do_train:
            output_eval_filename = "eval_examples_test"

        dev_dict = {'sentence': text, 'label_target': target, 'label_prediction': predicted}
        keys = ['sentence', 'label_target', 'label_prediction']
        output_examples_file = os.path.join(args.output_dir, output_eval_filename + ".tsv")
        file_test = open(output_examples_file, 'wt')
        dict_writer = csv.writer(file_test, delimiter='\t')
        dict_writer.writerow(keys)
        r = zip(*dev_dict.values())
        for d in r:
            dict_writer.writerow(d)

        # Mismatched example sentences and original and predicted labels
        dev_dict = {'sentence': text_mismatch, 'label_target': target_mismatch, 'label_prediction': predicted_mismatch}
        keys = ['sentence', 'label_target', 'label_prediction']
        output_examples_file = os.path.join(args.output_dir, output_eval_filename + "_mismatch.tsv")
        file_test = open(output_examples_file, 'wt')
        dict_writer = csv.writer(file_test, delimiter='\t')
        dict_writer.writerow(keys)
        r = zip(*dev_dict.values())
        for d in r:
            dict_writer.writerow(d)